# Create CSV file for Review data

In [1]:
from amazon_scraper import AmazonScraper
scraper = AmazonScraper()
asin = "B09C6HGX1W"  # Example ASIN

product = scraper.scrape_product_reviews(asin)
if product:
    print(f"Product: {product.name}")
    print(f"Overall Rating: {product.overall_rating}")
    print(f"Total Reviews: {product.total_review_count}")
    print(f"\nFirst {len(product.review_list)} reviews:")
    for review in product.review_list:
        print(f"\nReview by {review.country} on {review.date}")
        print(f"Rating: {review.rating}")
        print(f"Title: {review.title}")
        print(f"Verified Purchase: {review.verified_purchase}")
        print(f"Found Helpful: {review.found_helpful}")

Response status: 200
Response length: 117712
HTML length: 117712
Product name element not found
Rating element not found
Review count element not found
Found 0 review elements
Product: 
Overall Rating: 0.0
Total Reviews: 0

First 0 reviews:
